In [ ]:
# run this cell
from datascience import *
from pandas import read_stata
import numpy as np

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

data = Table.read_table('***Name of your data table***')
data

# Income
This week we are going to explore income inequality in your data.

For most of you, income data is in a separate data file.  Import this data file from Stata.

In [ ]:
# importing
employment =  Table.from_df(read_stata(***File Name***, convert_categoricals=False).fillna(-99))
employment

In [ ]:
# renaming
employment_vars = [***Current Variable Names***]
new_employment_vars = [***New Variable Names***]
employment_renamed = employment.select(employment_vars).relabel(employment_vars, new_employment_vars)

We want to have a total yearly income for each person's job.  Make a new column that converts the payment to yearly payment.  To do this, you will use the time frame about which the question was asked: income daily, biweekly, monthly, yearly, etc. 

In [ ]:
# you are going to want to change the return values in this function to fit your survey
# feel free to add as many elif clauses as necessary
def income_time_code(code):
    '''Converts the CODE to a constant for annualization of the income
    For example, if '1' corresponds to hour, we will convert it to a full time job.
    1 -- hour (would return 40*52 [40 hours per week, 52 weeks per year])
    2 -- day  (returns 5*52 [5 days a week, 52 weeks per year])
    3 -- week (returns 52 [52 weeks per year])
    4 -- month (returns 12 [12 months per year])'''
    code = int(code)
    if code == ***time code***:
        return ***conversion rate***
    elif code == ***time code***:
        return ***conversion rate***
    elif code == ***time code***:
        return ***conversion rate***
    elif code == ***time code***:
        return ***conversion rate***

In [ ]:
# time_coef should be an array of the values that must be multiplied to your income 
time_coef = employment_renamed.with_column('Time Coef', ***Apply the function that you defined in the previous cell***)

# A table with the bad values  filtered out
actual_data = time_coef.where(time_coef.apply(type, 'Time Coef') == int)

# Now multiply those time unit column and the column of their
# last paycheck together to get their annualized income
annualized = actual_data.with_column('Annualized', ***How do you get the values of this column?***)

employment_renamed = annualized
employment_renamed

Create the master ID for your employment_renamed table. The append_master_id function that you used in Lab 1 is below if you need to use it.

In [ ]:
def append_master_id(table, household_id_label="Household ID", individual_id_label="Individual ID"):

    #Fill in value of n. Should be 1 more than the number of digits of the largest number in the Household ID column.
    n = 3
    
    household_col = table[household_id_label]
    individual_col = table[individual_id_label]
    master_col = []
    for household_id, individual_id in zip(household_col, individual_col):
        master_col.append(household_id * 10**n + individual_id)
    return table.with_column('Master ID', master_col)

In [ ]:
# Make sure to drop unnecessary columns
employment_renamed = ***employment_renamed with a master id column***

Some people may have multiple jobs.  Combine incomes so we only have one observation for each person.

In [ ]:
# This line will show us how many individuals have more than one job
employment_renamed.group('Master ID').sort('count', descending = True)

In [ ]:
# HINT: group can take additional arguments, look at the documentation to see how to implement the second argument
# Once again, make sure to drop unnecessary columns
final_employment = 
final_employment

Join this new table ("Master ID" & "Yearly Income") to your main data table.  We don't want to lose the rows that do not have a master id in the final_employment table, so use join_keeping_all_main_table_rows. After you join,  make sure you still have the right number of people in your data set!

In [ ]:
def join_keeping_all_main_table_rows(main_table, joined, fill_in_value=-99, combining_column_label='Master ID', using_labels='left'):
    #can change using_labels to 'outer' if you want to keep all values of joining column from both tables
    df_main_table = main_table.to_df()
    df_joined = joined.to_df()
    df_combined = df_main_table.merge(df_joined, on=combining_column_label, how=using_labels).fillna(fill_in_value)
    return Table.from_df(df_combined)

In [ ]:
data = 
data

Save your data, we will be using this in future labs!

In [ ]:
data.to_csv('***desired file name***')

<font color="Blue"> Item 1: Among those who are earning money, what is the average income earned monthly by the people in your data.  Convert this to dollars - if you can, use the conversion rates from the year of the survey.  

We want to calculate the household income per capita and append it our data table. In order to do that, we will have to calculate the total household income and total number of people for each household. We will also do this for households that do not have income earners.

In [ ]:
# First, add a column 'Household Size' to your data table, 
# which has the number of people in the household for each row
# hint: you may have this in the household composition lab (Lab 4)


In [ ]:
# Next, using your income_earners table, group by Household ID and find
# the sum of incomes for each household. Turn that into a table with two
# columns, 'Household ID' and 'Household Income'


In [ ]:
# Now combine the two tables using the join_keeping_all_main_table_rows
# function, and use these values for your arguments:
# main_table = your data table
# joined = your table with household incomes
# fill_in_value = 0 (households without any income earners have zero dollars)
# combining_column_label = 'Household ID'


In [ ]:
# Finally, you will create a new column, 'Household Income per Capita,' which is 
# equal to 'Household Income' divided by 'Household Size'


<font color="Blue"> Item 2: What is the average household income per capita? (Note: make sure your denominator is households, not individuals.) Convert to dollars.  Compare this to the World Bank's interational poverty line of $1.90 a day. 

Do this once for households with income earners, and then once for all households.

<font color="Blue"> Item 3: What percentage of households live below your country's poverty line (you will have to look this up - could be different from the World Bank poverty line.)

Do this once for households with income earners, and then once for all households.

## Income inequality - the Lorenz Curve & Gini Coefficent
Examining the poverty line is one way to consider income inquality.  Another way is to look at concentration of wealth.  The Lorenz curve is a graphical illustration of this and the Gini Coefficient is a numeric representation.  The Lorenz curve lines up the population from poorest to richest on the x axis, and then the y-axis graphs cumulative income of all the people poorer than the one on the x axis.  To make these comparable across countries, these are translated into percentages.  (0,0) and (100,100) are always the end points of the curve: 0% of the people own nothing while 100% of the people own 100% of the wealth.  

Let's line up our individual income earners to look at inequality just among income earners.  Let's start with a fresh table, with the column of the income.  Let's put the table of incomes in order from lowest to highest.  Then let's make a new column, that adds up all incomes above the row.  Here's an example:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-baqh">Income</th>
    <th class="tg-baqh">Cumulative Income</th>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">3</td>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">6</td>
  </tr>
  <tr>
    <td class="tg-baqh">4</td>
    <td class="tg-baqh">10</td>
  </tr>
  <tr>
    <td class="tg-baqh">7</td>
    <td class="tg-baqh">17</td>
  </tr>
  <tr>
    <td class="tg-baqh">8</td>
    <td class="tg-baqh">26</td>
  </tr>
</table>

In [ ]:
# First, you are going to want to get your incomes in order
# this should be a list or an array
ordered_incomes = 

In [ ]:
# This for loop creates a list called cumulative_step, which stores the values for the 'Cumulative Income' column
total = 0
cumulative_step = []
for x in ordered_incomes:
    total = x + total
    cumulative_step.append(total)

In [ ]:
lorenz_table = Table().with_columns(
    ['Income', ordered_incomes],
    ['Cumulative Income', cumulative_step])

Now we will need to turn these into percentages.  Go to the last row of the table.  This will tell you the total amount of income in the economy.  We will divide the Cumulative Income column by this number.  In the example, that is 25.  Although my table shows the percent sign, you do not have to include the percent sign in your table.  In fact, there will be one less step for graphing if you do not include the percent sign.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-baqh">Income</th>
    <th class="tg-baqh">Cumulative Income</th>
    <th class="tg-yw4l">Percentage Income</th>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">3</td>
    <td class="tg-yw4l">12%</td>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">6</td>
    <td class="tg-yw4l">24%</td>
  </tr>
  <tr>
    <td class="tg-baqh">4</td>
    <td class="tg-baqh">10</td>
    <td class="tg-yw4l">40%</td>
  </tr>
  <tr>
    <td class="tg-baqh">7</td>
    <td class="tg-baqh">17</td>
    <td class="tg-yw4l">68%</td>
  </tr>
  <tr>
    <td class="tg-baqh">8</td>
    <td class="tg-baqh">25</td>
    <td class="tg-yw4l">100%</td>
  </tr>
</table>


In [ ]:
# This for loop creates a list called percents, which stores the values for the 'Percentage Income' column
# Define total_income, which is the sum of all of the incomes in the table
total_income = 
percents = []
for x in cumulative_step:
    percent = x / total_income
    percents.append(percent)

In [ ]:
# Add 'percents' to 'lorenz_table'
lorenz_table = 

Now we need to put the population in terms of percentage.  Add a new column that counts the income earners.  As in this table, there may be some people that have the same income.  It does not matter which order these go in.  Again, go to the last row and divide the count column by the total number of income earners.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-baqh">Income</th>
    <th class="tg-baqh">Cumulative Income</th>
    <th class="tg-yw4l">Percentage Income</th>
    <th class="tg-yw4l">Count</th>
    <th class="tg-yw4l">Percentage Population</th>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">3</td>
    <td class="tg-yw4l">12%</td>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">20%</td>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">6</td>
    <td class="tg-yw4l">24%</td>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">40%</td>
  </tr>
  <tr>
    <td class="tg-baqh">4</td>
    <td class="tg-baqh">10</td>
    <td class="tg-yw4l">40%</td>
    <td class="tg-yw4l">3</td>
    <td class="tg-yw4l">60%</td>
  </tr>
  <tr>
    <td class="tg-baqh">7</td>
    <td class="tg-baqh">17</td>
    <td class="tg-yw4l">68%</td>
    <td class="tg-yw4l">4</td>
    <td class="tg-yw4l">80%</td>
  </tr>
  <tr>
    <td class="tg-baqh">8</td>
    <td class="tg-baqh">26</td>
    <td class="tg-yw4l">100%</td>
    <td class="tg-yw4l">5</td>
    <td class="tg-yw4l">100%</td>
  </tr>
</table>

In [ ]:
# This for loop creates two lists, spot and pop_percent, which
#     contain the cumulative number of people and the cumulative population percentage
# Define total_pop, which the number of people in this table
total_pop = 
count = 0
spot = []
pop_percent = []
for i in ordered_incomes:
    count += 1
    spot.append(count)
    pop_percent.append(count / total_pop)

We are missing the 0,0 row at the begining.  Add this row in.  You have not been using percentage signs in your table, so you don't need to add them in for the these zeros either.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-baqh">Income</th>
    <th class="tg-baqh">Cumulative Income</th>
    <th class="tg-yw4l">Percentage Income</th>
    <th class="tg-yw4l">Count</th>
    <th class="tg-yw4l">Percentage Population</th>
  </tr>
  <tr>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l">0%</td>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l">0%</td>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">3</td>
    <td class="tg-yw4l">12%</td>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">20%</td>
  </tr>
  <tr>
    <td class="tg-baqh">3</td>
    <td class="tg-baqh">6</td>
    <td class="tg-yw4l">24%</td>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">40%</td>
  </tr>
  <tr>
    <td class="tg-baqh">4</td>
    <td class="tg-baqh">10</td>
    <td class="tg-yw4l">40%</td>
    <td class="tg-yw4l">3</td>
    <td class="tg-yw4l">60%</td>
  </tr>
  <tr>
    <td class="tg-baqh">7</td>
    <td class="tg-baqh">17</td>
    <td class="tg-yw4l">68%</td>
    <td class="tg-yw4l">4</td>
    <td class="tg-yw4l">80%</td>
  </tr>
  <tr>
    <td class="tg-baqh">8</td>
    <td class="tg-baqh">26</td>
    <td class="tg-yw4l">100%</td>
    <td class="tg-yw4l">5</td>
    <td class="tg-yw4l">100%</td>
  </tr>
</table>

In [ ]:
# Because we will only be using the columns 'Population Income' and 'Percentage Population,'
# it does not matter what you fill in for the other column values of the inserted row


You are ready to graph the Lorenz curve! 
<font color="Blue"> Item 4: Make a line graph with percentage population on the x axis and percentage income on the y axis.

In [ ]:
x_values = lorenz_table['Percentage Population']
y_values = lorenz_table['Percentage Income']

plots.xlabel('Income Percentage')
plots.ylabel('Population Percentage')
plots.title('Lorenz Curve')

plots.scatter(x_values,y_values)

<font color="Blue"> Item 5: Calculate the Gini coefficient.
<font color="Black">The Gini coefficient is the area between the line and the 45% line.  It is the sum of the differences between Percent Population and Percent Income divided by the sum of Percent Population.  This number is between 0 and 1.  Consider if a smaller number represents more inequality and what the relationship of the formula to the graph is.  

$$ \frac{\Sigma_{i=1}^{N} (\% Pop_i - \% Inc_i)}{\Sigma_{i=1}^{N} \% Pop_i}$$

Note that if there were perfect equality, $\% Pop_i = \% Inc_i$.

<font color="Blue"> Items 4a & 5a: Repeat Items 4 & 5, but instead of using individuals who earn, use all individuals, using household income per capita as the income for each person in the household.  Is there more or less inquality among households or among individual earners as measured by the Lorenz curve & Gini coefficient?

<font color="Blue"> Item 6: We use Household Income Per Capita, assuming that everyone in the household consumes and equal share of the earnings.  Do you think this is accurate?  If not, and we could accurately measure how much each person consumed in the household, how would using individual consumption instead of average income change our measures of inequality?